In [1]:
import pandas as pd
import numpy as nb
import openpyxl as op
import seaborn as sn
from datetime import datetime as dt
import pandas as pd
import numpy as np
import scipy
import statsmodels
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
import seaborn as sns
import matplotlib.pyplot as plt

# Loading Data

In [2]:
incident_df = pd.read_excel('C:/Users/brigh/OneDrive/Desktop/Detail_Incident Scrubbed.xlsx')

# wrangling of the data

In [4]:
incident_df2 = incident_df.set_index('Incident ID')

In [5]:
incident_df2_top = incident_df.head()

In [6]:
incident_df2

,Priority,Category,# Reassignments,Open Time,Reopen Time,Resolved Time,Close Time,Handle Time (Hours),CI Name,CI Type,CI Subtype,Service Component WBS,KM number,# Related Interactions
Incident ID,,,,,,,,,,,,,,
IM0034604,1,incident,0.0,06/02/2014 09:40:35,12/02/2014 12:00:18,12/02/2014 12:01:14,12/02/2014 12:01:18,30.0,WBA000058,application,Web Based Application,WBS000073,KM0002043,Software
IM0035403,1,incident,1.0,10/02/2014 09:20:28,12/02/2014 12:02:45,12/02/2014 12:03:01,12/02/2014 12:03:05,23.0,SUB000523,subapplication,Web Based Application,WBS000073,KM0000818,Data
IM0037024,1,incident,0.0,15/02/2014 10:29:49,17/02/2014 11:51:27,17/02/2014 11:54:21,17/02/2014 11:54:24,1.0,WBA000068,application,Web Based Application,WBS000171,KM0000697,Hardware
IM0000944,2,incident,5.0,27/09/2013 08:52:38,NaN,01/10/2013 18:20:07,01/10/2013 18:23:03,24.0,WBA000148,application,Web Based Application,WBS000172,KM0000697,Other
IM0001225,2,incident,1.0,01/10/2013 08:18:24,NaN,NaN,01/10/2013 15:34:38,7.0,SBA000759,application,Server Based Application,WBS000296,KM0001031,Software
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IM0047033,5,request for information,0.0,31/03/2014 12:44:51,NaN,31/03/2014 14:10:43,31/03/2014 14:10:45,1.0,WBA000124,application,Web Based Application,WBS000088,KM0000611,Other
IM0047040,5,request for information,0.0,31/03/2014 13:42:26,NaN,31/03/2014 15:41:57,31/03/2014 15:42:01,1.0,OVR000033,no type,no subtype,WBS000118,KM0001172,Operator error
IM0047048,5,incident,1.0,31/03/2014 15:04:27,NaN,31/03/2014 16:31:53,31/03/2014 16:32:33,1.0,WBA000124,application,Web Based Application,WBS000088,KM0000611,Other


### Looking at the Date Columns, the order and types of years vary between a 2 digit year and a 4 digit year so using a script I found at: https://stackoverflow.com/questions/68355770/convert-multiple-date-formats-to-datetime-in-pandas I was able to change them all to Year/Month/Day

In [12]:
incident_df2['Open Time'] = pd.to_datetime(incident_df2['Open Time'], errors='coerce')
incident_df2['Reopen Time'] = pd.to_datetime(incident_df2['Reopen Time'], errors='coerce')
incident_df2['Resolved Time'] = pd.to_datetime(incident_df2['Resolved Time'], errors='coerce')
incident_df2['Close Time'] = pd.to_datetime(incident_df2['Close Time'], errors='coerce')

In [13]:
incident_df2

,Priority,Category,# Reassignments,Open Time,Reopen Time,Resolved Time,Close Time,Handle Time (Hours),CI Name,CI Type,CI Subtype,Service Component WBS,KM number,# Related Interactions
Incident ID,,,,,,,,,,,,,,
IM0034604,1,incident,0.0,2014-06-02 09:40:35,2014-12-02 12:00:18,2014-12-02 12:01:14,2014-12-02 12:01:18,30.0,WBA000058,application,Web Based Application,WBS000073,KM0002043,Software
IM0035403,1,incident,1.0,2014-10-02 09:20:28,2014-12-02 12:02:45,2014-12-02 12:03:01,2014-12-02 12:03:05,23.0,SUB000523,subapplication,Web Based Application,WBS000073,KM0000818,Data
IM0037024,1,incident,0.0,2014-02-15 10:29:49,2014-02-17 11:51:27,2014-02-17 11:54:21,2014-02-17 11:54:24,1.0,WBA000068,application,Web Based Application,WBS000171,KM0000697,Hardware
IM0000944,2,incident,5.0,2013-09-27 08:52:38,NaT,2013-01-10 18:20:07,2013-01-10 18:23:03,24.0,WBA000148,application,Web Based Application,WBS000172,KM0000697,Other
IM0001225,2,incident,1.0,2013-01-10 08:18:24,NaT,NaT,2013-01-10 15:34:38,7.0,SBA000759,application,Server Based Application,WBS000296,KM0001031,Software
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IM0047033,5,request for information,0.0,2014-03-31 12:44:51,NaT,2014-03-31 14:10:43,2014-03-31 14:10:45,1.0,WBA000124,application,Web Based Application,WBS000088,KM0000611,Other
IM0047040,5,request for information,0.0,2014-03-31 13:42:26,NaT,2014-03-31 15:41:57,2014-03-31 15:42:01,1.0,OVR000033,no type,no subtype,WBS000118,KM0001172,Operator error
IM0047048,5,incident,1.0,2014-03-31 15:04:27,NaT,2014-03-31 16:31:53,2014-03-31 16:32:33,1.0,WBA000124,application,Web Based Application,WBS000088,KM0000611,Other


In [14]:
incident_df2.isnull().sum()

Priority                      0
Category                      0
# Reassignments               1
Open Time                     0
Reopen Time               44322
Resolved Time              1780
Close Time                    0
Handle Time (Hours)           1
CI Name                       0
CI Type                       0
CI Subtype                    0
Service Component WBS         0
KM number                     0
# Related Interactions      459
dtype: int64

# Dropping columns

In [15]:
incident_df3 = incident_df2.drop('Reopen Time', 1)

C:\Users\brigh\AppData\Local\Temp/ipykernel_2820/679424260.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  incident_df3 = incident_df2.drop('Reopen Time', 1)


### Below shows where the Columns have been removed

In [16]:
incident_df3

,Priority,Category,# Reassignments,Open Time,Resolved Time,Close Time,Handle Time (Hours),CI Name,CI Type,CI Subtype,Service Component WBS,KM number,# Related Interactions
Incident ID,,,,,,,,,,,,,
IM0034604,1,incident,0.0,2014-06-02 09:40:35,2014-12-02 12:01:14,2014-12-02 12:01:18,30.0,WBA000058,application,Web Based Application,WBS000073,KM0002043,Software
IM0035403,1,incident,1.0,2014-10-02 09:20:28,2014-12-02 12:03:01,2014-12-02 12:03:05,23.0,SUB000523,subapplication,Web Based Application,WBS000073,KM0000818,Data
IM0037024,1,incident,0.0,2014-02-15 10:29:49,2014-02-17 11:54:21,2014-02-17 11:54:24,1.0,WBA000068,application,Web Based Application,WBS000171,KM0000697,Hardware
IM0000944,2,incident,5.0,2013-09-27 08:52:38,2013-01-10 18:20:07,2013-01-10 18:23:03,24.0,WBA000148,application,Web Based Application,WBS000172,KM0000697,Other
IM0001225,2,incident,1.0,2013-01-10 08:18:24,NaT,2013-01-10 15:34:38,7.0,SBA000759,application,Server Based Application,WBS000296,KM0001031,Software
...,...,...,...,...,...,...,...,...,...,...,...,...,...
IM0047033,5,request for information,0.0,2014-03-31 12:44:51,2014-03-31 14:10:43,2014-03-31 14:10:45,1.0,WBA000124,application,Web Based Application,WBS000088,KM0000611,Other
IM0047040,5,request for information,0.0,2014-03-31 13:42:26,2014-03-31 15:41:57,2014-03-31 15:42:01,1.0,OVR000033,no type,no subtype,WBS000118,KM0001172,Operator error
IM0047048,5,incident,1.0,2014-03-31 15:04:27,2014-03-31 16:31:53,2014-03-31 16:32:33,1.0,WBA000124,application,Web Based Application,WBS000088,KM0000611,Other


### Create a new column/variable to show how much Time each ticket took to close

In [17]:
incident_df3["Time Elapse"] = incident_df3['Close Time'] - incident_df3['Open Time']

In [18]:
incident_df3["Time Elapse"]

Incident ID
IM0034604     183 days 02:20:43
IM0035403      61 days 02:42:37
IM0037024       2 days 01:24:35
IM0000944   -260 days +09:30:25
IM0001225       0 days 07:16:14
                    ...        
IM0047033       0 days 01:25:54
IM0047040       0 days 01:59:35
IM0047048       0 days 01:28:06
IM0047050       0 days 01:24:59
IM0047055       0 days 00:04:18
Name: Time Elapse, Length: 46606, dtype: timedelta64[ns]

Created 4 variables to show by incident (will be great to use when visualizing patterns)

1. What Day of the Week (DOW) was opened
2. What Day of the Week (DOW) was closed
3. What Week of the Year was opened
4. What Week of the Year was closed

0 is Monday, Sunday is 7

In [19]:
incident_df3["Open DOW"] = incident_df3['Open Time'].dt.isocalendar().week
incident_df3["Close DOW"] = incident_df3['Close Time'].dt.isocalendar().week
incident_df3["Open Week"] = incident_df3['Open Time'].dt.isocalendar().week
incident_df3["Close Week"] = incident_df3['Close Time'].dt.isocalendar().week

In [20]:
incident_df3

,Priority,Category,# Reassignments,Open Time,Resolved Time,Close Time,Handle Time (Hours),CI Name,CI Type,CI Subtype,Service Component WBS,KM number,# Related Interactions,Time Elapse,Open DOW,Close DOW,Open Week,Close Week
Incident ID,,,,,,,,,,,,,,,,,,
IM0034604,1,incident,0.0,2014-06-02 09:40:35,2014-12-02 12:01:14,2014-12-02 12:01:18,30.0,WBA000058,application,Web Based Application,WBS000073,KM0002043,Software,183 days 02:20:43,23,49,23,49
IM0035403,1,incident,1.0,2014-10-02 09:20:28,2014-12-02 12:03:01,2014-12-02 12:03:05,23.0,SUB000523,subapplication,Web Based Application,WBS000073,KM0000818,Data,61 days 02:42:37,40,49,40,49
IM0037024,1,incident,0.0,2014-02-15 10:29:49,2014-02-17 11:54:21,2014-02-17 11:54:24,1.0,WBA000068,application,Web Based Application,WBS000171,KM0000697,Hardware,2 days 01:24:35,7,8,7,8
IM0000944,2,incident,5.0,2013-09-27 08:52:38,2013-01-10 18:20:07,2013-01-10 18:23:03,24.0,WBA000148,application,Web Based Application,WBS000172,KM0000697,Other,-260 days +09:30:25,39,2,39,2
IM0001225,2,incident,1.0,2013-01-10 08:18:24,NaT,2013-01-10 15:34:38,7.0,SBA000759,application,Server Based Application,WBS000296,KM0001031,Software,0 days 07:16:14,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IM0047033,5,request for information,0.0,2014-03-31 12:44:51,2014-03-31 14:10:43,2014-03-31 14:10:45,1.0,WBA000124,application,Web Based Application,WBS000088,KM0000611,Other,0 days 01:25:54,14,14,14,14
IM0047040,5,request for information,0.0,2014-03-31 13:42:26,2014-03-31 15:41:57,2014-03-31 15:42:01,1.0,OVR000033,no type,no subtype,WBS000118,KM0001172,Operator error,0 days 01:59:35,14,14,14,14
IM0047048,5,incident,1.0,2014-03-31 15:04:27,2014-03-31 16:31:53,2014-03-31 16:32:33,1.0,WBA000124,application,Web Based Application,WBS000088,KM0000611,Other,0 days 01:28:06,14,14,14,14


In [ ]:
##CI Merge columns 

In [35]:
incident_df3['CI Merge'] = incident_df3['CI Type'] + " " + incident_df3['CI Subtype'].map(str)

In [39]:
incident_df3.head()

,Priority,Category,# Reassignments,Open Time,Resolved Time,Close Time,Handle Time (Hours),CI Name,CI Type,CI Subtype,Service Component WBS,KM number,# Related Interactions,Time Elapse,Open DOW,Close DOW,Open Week,Close Week,CI Merge
Incident ID,,,,,,,,,,,,,,,,,,,
IM0034604,1,incident,0.0,2014-06-02 09:40:35,2014-12-02 12:01:14,2014-12-02 12:01:18,30.0,WBA000058,application,Web Based Application,WBS000073,KM0002043,Software,183 days 02:20:43,23,49,23,49,application Web Based Application
IM0035403,1,incident,1.0,2014-10-02 09:20:28,2014-12-02 12:03:01,2014-12-02 12:03:05,23.0,SUB000523,subapplication,Web Based Application,WBS000073,KM0000818,Data,61 days 02:42:37,40,49,40,49,subapplication Web Based Application
IM0037024,1,incident,0.0,2014-02-15 10:29:49,2014-02-17 11:54:21,2014-02-17 11:54:24,1.0,WBA000068,application,Web Based Application,WBS000171,KM0000697,Hardware,2 days 01:24:35,7,8,7,8,application Web Based Application
IM0000944,2,incident,5.0,2013-09-27 08:52:38,2013-01-10 18:20:07,2013-01-10 18:23:03,24.0,WBA000148,application,Web Based Application,WBS000172,KM0000697,Other,-260 days +09:30:25,39,2,39,2,application Web Based Application
IM0001225,2,incident,1.0,2013-01-10 08:18:24,NaT,2013-01-10 15:34:38,7.0,SBA000759,application,Server Based Application,WBS000296,KM0001031,Software,0 days 07:16:14,2,2,2,2,application Server Based Application
